In [1]:
import re
import numpy as np
import pandas as pd
import json

In [2]:
from libcg.util.commentutil import CommentUtil
from libcg.util.fileutil import FileUtil
from libcg.util.treeutil import TreeUtil

In [3]:
def pushToTHMTREE(theme, type, subid, data):
    global tutil
    evalRules = {}
    for index, row in data.iterrows():
        subid = row['SubThemeID']
        num = row['Num']
        others = tutil.futil.convertDfToJson(row, data.columns.tolist(), ['SubThemeID', 'Num'])
        tutil.trees['THMTREE'], evalRules = tutil.addToTREE(tutil.trees['THMTREE'], 0, [theme, type, subid, [{num:others}]], 'THMTREE', tutil.rules['UNIQ'], evalRules)

In [4]:
def pushToGRDTREE(grid, type, subid, data):
    global tutil
    evalRules = {}
    for index, row in data.iterrows():
        subid = row['SubGridID']
        num = row['Num']
        others = tutil.futil.convertDfToJson(row, data.columns.tolist(), ['SubGridID', 'Num'])
        tutil.trees['GRDTREE'], evalRules = tutil.addToTREE(tutil.trees['GRDTREE'], 0, [grid, type, subid, [{num:others}]], 'GRDTREE', tutil.rules['UNIQ'], evalRules)

In [5]:
def pushToDATTREE(dataid, type, subid, data):
    global tutil
    evalRules = {}
    for index, row in data.iterrows():
        subid = row['SubDataID']
        num = row['Num']
        others = tutil.futil.convertDfToJson(row, data.columns.tolist(), ['SubDataID', 'Num'])
        tutil.trees['DATTREE'], evalRules = tutil.addToTREE(tutil.trees['DATTREE'], 0, [dataid, type, subid, [{num:others}]], 'DATTREE', tutil.rules['UNIQ'], evalRules)

In [6]:
def getTheme(theme):
    if type(theme) is str:
        global tutil
        evalRules = {}
        if (theme not in tutil.trees['THMTREE'].keys()):
            THEMELIST = tutil.futil.readSetFromSheet('Themes', 'ThemeID', theme)    
            legendRow = THEMELIST[THEMELIST['Type']=='Legend']
            titleRow = THEMELIST[THEMELIST['Type']=='Title']
            legend = legendRow.iloc[0]['SubThemeID']
            title = titleRow.iloc[0]['SubThemeID']
            colourset = titleRow.iloc[0]['SubThemeID']
            THEMELEGLIST = tutil.futil.readSetFromSheet('Themes-Legend', 'SubThemeID', legend)
            THEMETITLELIST = tutil.futil.readSetFromSheet('Themes-Title', 'SubThemeID', title)
            THEMECOLOURSETLIST = tutil.futil.readSetFromSheet('Themes-Colourset', 'SubThemeID', colourset)
            if colourset:
                pushToTHMTREE(theme, 'Colourset', colourset, THEMECOLOURSETLIST)
                tutil.trees['THMSUBTHM'], evalRules = tutil.addToTREE(tutil.trees['THMSUBTHM'], 0, [theme, [{'Colourset':colourset}]], 'THMSUBTHM', tutil.rules['UNIQ'], evalRules)
            pushToTHMTREE(theme, 'Legend', legend, THEMELEGLIST)
            pushToTHMTREE(theme, 'Title', title, THEMETITLELIST)
            tutil.trees['THMSUBTHM'], evalRules = tutil.addToTREE(tutil.trees['THMSUBTHM'], 0, [theme, [{'Legend':legend}]], 'THMSUBTHM', tutil.rules['UNIQ'], evalRules)
            tutil.trees['THMSUBTHM'], evalRules = tutil.addToTREE(tutil.trees['THMSUBTHM'], 0, [theme, [{'Title':title}]], 'THMSUBTHM', tutil.rules['UNIQ'], evalRules)

In [7]:
def getData(data):
    if type(data) is str:
        global tutil
        evalRules = {}
        if (data not in tutil.trees['DATTREE'].keys()):
            DATLIST = tutil.futil.readSetFromSheet('Data', 'DataID', data)
            dataPointRow = DATLIST[DATLIST['Type']=='Datapoint']
            fieldListRow = DATLIST[DATLIST['Type']=='Fieldlist']
            uriRow = DATLIST[DATLIST['Type']=='URI']
            if len(uriRow)>0:
                uriRow = DATLIST[DATLIST['Type']=='URI']
                uriList = uriRow.iloc[0]['SubDataID']
                DATAURILIST = tutil.futil.readSetFromSheet('Data-URI', 'SubDataID', uriList)
                pushToDATTREE(data, 'URI', uriList, DATAURILIST)
                tutil.trees['DATSUBDAT'], evalRules = tutil.addToTREE(tutil.trees['DATSUBDAT'], 0, [data, [{'URI':uriList}]], 'DATSUBDAT', tutil.rules['UNIQ'], evalRules)
            dataPoint = dataPointRow.iloc[0]['SubDataID']
            fieldList = fieldListRow.iloc[0]['SubDataID']
            DATADPLIST = tutil.futil.readSetFromSheet('Data-Datapoint', 'SubDataID', dataPoint)
            DATAFLLIST = tutil.futil.readSetFromSheet('Data-Fieldlist', 'SubDataID', fieldList)            
            pushToDATTREE(data, 'Datapoint', dataPoint, DATADPLIST)
            pushToDATTREE(data, 'Fieldlist', fieldList, DATAFLLIST)
            tutil.trees['DATSUBDAT'], evalRules = tutil.addToTREE(tutil.trees['DATSUBDAT'], 0, [data, [{'Datapoint':dataPoint}]], 'DATSUBDAT', tutil.rules['UNIQ'], evalRules)
            tutil.trees['DATSUBDAT'], evalRules = tutil.addToTREE(tutil.trees['DATSUBDAT'], 0, [data, [{'Fieldlist':fieldList}]], 'DATSUBDAT', tutil.rules['UNIQ'], evalRules)            

In [8]:
def getGrid(data):
    if type(data) is str:
        global tutil
        evalRules = {}
        if (data not in tutil.trees['GRDTREE'].keys()):
            DATLIST = tutil.futil.readSetFromSheet('Grid', 'GridID', data)
            configRow = DATLIST[DATLIST['Type']=='Config']
            fieldListRow = DATLIST[DATLIST['Type']=='FieldHdrlist']
            formatListRow = DATLIST[DATLIST['Type']=='Formatlist']
            configs = configRow.iloc[0]['SubGridID']
            fieldList = fieldListRow.iloc[0]['SubGridID']
            formatList = formatListRow.iloc[0]['SubGridID']
            DATACFGLIST = tutil.futil.readSetFromSheet('Grid-Config', 'SubGridID', configs)
            DATAFLLIST = tutil.futil.readSetFromSheet('Grid-FieldHdrlist', 'SubGridID', fieldList)
            DATAFMTLIST = tutil.futil.readSetFromSheet('Grid-Formatlist', 'SubGridID', formatList)
            pushToGRDTREE(data, 'Config', configs, DATACFGLIST)
            pushToGRDTREE(data, 'FieldHdrlist', fieldList, DATAFLLIST)
            pushToGRDTREE(data, 'Formatlist', fieldList, DATAFMTLIST)
            tutil.trees['GRDSUBDET'], evalRules = tutil.addToTREE(tutil.trees['GRDSUBDET'], 0, [data, [{'Config':configs}]], 'GRDSUBDET', tutil.rules['UNIQ'], evalRules)
            tutil.trees['GRDSUBDET'], evalRules = tutil.addToTREE(tutil.trees['GRDSUBDET'], 0, [data, [{'FieldHdrlist':fieldList}]], 'GRDSUBDET', tutil.rules['UNIQ'], evalRules)
            tutil.trees['GRDSUBDET'], evalRules = tutil.addToTREE(tutil.trees['GRDSUBDET'], 0, [data, [{'Formatlist':formatList}]], 'GRDSUBDET', tutil.rules['UNIQ'], evalRules)

In [9]:
def funcResolve(tutil, param):
    return tutil.resolveReserved(tutil.trees['THMTREE'], tutil.trees['THMSUBTHM'], param[0], param[1], param[2], [False, False], False, False)

In [10]:
def funcResolveData(tutil, param):
    return tutil.resolveReserved(tutil.trees['DATTREE'], tutil.trees['DATSUBDAT'], param[0], param[1], param[2], [False, False], False, False)

In [11]:
def funcResolveGrid(tutil, param):
    return tutil.resolveReserved(tutil.trees['GRDTREE'], tutil.trees['GRDSUBDET'], param[0], param[1], param[2], [False, False], False, False)

In [12]:
def funcResolveDataEsc(tutil, param):
    #print(json.dumps({'---> funcResolveDataEsc': param }, sort_keys=True, indent=4))
    result = tutil.resolveReserved(tutil.trees['DATTREE'], tutil.trees['DATSUBDAT'], param[0], param[1], param[2], [False, False], False, False)
    return result

In [13]:
def funcResolveGridEsc(tutil, param):
    #print(json.dumps({'---> funcResolveGridEsc': param }, sort_keys=True, indent=4))
    result = tutil.resolveReserved(tutil.trees['GRDTREE'], tutil.trees['GRDSUBDET'], param[0], param[1], param[2], [False, False], False, False)
    return result

In [14]:
def funcIterateData(tutil, param):
    result = param[1]
    if param[2]:
        #print(json.dumps({'---> funcIterateData': param }, sort_keys=True, indent=4))
        mainID = param[0]
        block = param[1]
        extractPatternList = param[2]
        regexPat = param[3]
        resolvedStmt={}
        markers = re.findall(r"{}".format("(##ITERATEDATA\([^#]+\)##)"), block)
        #print("markers", markers)
        for stmtIdx, patStmt in enumerate(extractPatternList):
            RESERVED = patStmt[0]
            paramList = patStmt[1].split(",")
            TYPE = paramList[0]
            STMT = paramList[1]
            subId = tutil.trees['DATSUBDAT'][mainID][TYPE]
            varList = json.loads(tutil.rules['LEVEL_3_RULES']['Replace']['FieldDelimiter'](paramList[2]))
            #print(json.dumps({'varList':varList}, sort_keys=True, indent=4))
            fieldlist = tutil.trees['DATTREE'][mainID][TYPE][subId]
            #print(json.dumps({'fieldlist':fieldlist}, sort_keys=True, indent=4))
            #value = resolveParams([KV[3], value])
            subResult = ""
            for idx, field in fieldlist.items():
                subsList = []
                for vIdx, v in enumerate(varList):
                    #subsList[vIdx+1]=field[v]
                    subsList.append(tutil.funcNullable(field[v]))
                newSTMT = tutil.resolveParams([json.dumps(subsList), STMT])
                subResult = "{}\n{}".format(subResult, newSTMT)
            resolvedStmt[stmtIdx] = subResult
        for mIdx, m in enumerate(markers):
            result = tutil.replacePattern(result, m, resolvedStmt[mIdx], [True, False])
        #print("resolvedStmt", resolvedStmt)
    #result = tutil.resolveReserved(DATTREE, DATSUBDAT, param[0], param[1], param[2], [False, False], False, False)
    return result

In [15]:
def funcIterateGrid(tutil, param):
    result = param[1]
    if param[2]:
        #print(json.dumps({'---> funcIterateGrid': param }, sort_keys=True, indent=4))
        mainID = param[0]
        block = param[1]
        extractPatternList = param[2]
        regexPat = param[3]
        resolvedStmt={}
        markers = re.findall(r"{}".format("(##ITERATEGRID\([^#]+\)##)"), block)
        #print("markers", markers)
        for stmtIdx, patStmt in enumerate(extractPatternList):
            RESERVED = patStmt[0]
            paramList = patStmt[1].split(",")
            TYPE = paramList[0]
            STMT = paramList[1]
            subId = tutil.trees['GRDSUBDET'][mainID][TYPE]
            varList = json.loads(tutil.rules['LEVEL_3_RULES']['Replace']['FieldDelimiter'](paramList[2]))
            fieldlist = tutil.trees['GRDTREE'][mainID][TYPE][subId]
            #value = resolveParams([KV[3], value])
            subResult = ""
            for idx, field in fieldlist.items():
                subsList = []
                for vIdx, v in enumerate(varList):
                    #subsList[vIdx+1]=field[v]
                    subsList.append(tutil.funcNullable(field[v]))
                newSTMT = tutil.resolveParams([json.dumps(subsList), STMT])
                subResult = "{}\n{}".format(subResult, newSTMT)
            resolvedStmt[stmtIdx] = subResult
        for mIdx, m in enumerate(markers):
            result = tutil.replacePattern(result, m, resolvedStmt[mIdx], [True, False])
        #print("resolvedStmt", resolvedStmt)
    #result = tutil.resolveReserved(GRDTREE, GRDSUBDET, param[0], param[1], param[2], [False, False], False, False)
    return result

In [16]:
def funcIterateThemes(tutil, param):
    result = param[1]
    if param[2]:
        #print(json.dumps({'---> funcIterateThemes': param }, sort_keys=True, indent=4))
        mainID = param[0]
        block = param[1]
        extractPatternList = param[2]
        regexPat = param[3]
        resolvedStmt={}
        markers = re.findall(r"{}".format("(##ITERATETHEMES\([^#]+\)##)"), block)
        #print("markers-->", markers)
        for stmtIdx, patStmt in enumerate(extractPatternList):
            RESERVED = patStmt[0]
            paramList = patStmt[1].split(",")
            TYPE = paramList[0]
            STMT = paramList[1]
            #print("TYPE[{}]MAINID[{}]-->OUTSIDE".format(TYPE,mainID))
            if TYPE in tutil.trees['THMTREE'][mainID]:
                subId = tutil.trees['THMSUBTHM'][mainID][TYPE]
                #print("paramList", paramList)
                varList = json.loads(tutil.rules['LEVEL_3_RULES']['Replace']['FieldDelimiter'](paramList[2]))
                #print("varList-->INSIDE ", varList)
                if subId in tutil.trees['THMSUBTHM'][mainID][TYPE]:
                    fieldlist = tutil.trees['THMTREE'][mainID][TYPE][subId]
                    #print("fieldlist-->", fieldlist)
                    #value = resolveParams([KV[3], value])
                    subResult = ""
                    for idx, field in fieldlist.items():
                        subsList = []
                        for vIdx, v in enumerate(varList):
                            #subsList[vIdx+1]=field[v]
                            subsList.append(tutil.funcNullable(field[v]))
                        newSTMT = tutil.resolveParams([json.dumps(subsList), STMT])
                        subResult = "{}\n{}".format(subResult, newSTMT)
                        resolvedStmt[stmtIdx] = subResult
        for mIdx, m in enumerate(markers):
            #print("resolvedStmt", resolvedStmt)
            result = tutil.replacePattern(result, m, resolvedStmt[mIdx], [True, False])
        #print("resolvedStmt", resolvedStmt)
    #result = tutil.resolveReserved(THMTREE, THMSUBTHM, param[0], param[1], param[2], [False, False], False, False)
    return result

In [17]:
def runMainSetup(cfgfile, sheetName, futil, tutil):
    cfg = pd.ExcelFile(cfgfile)
    df =  pd.read_excel(cfg, sheetName)
    tutil.getVar()
    for index, row in df.iterrows():
        tutil.getTemplate(row['CodeBlockID'])
        getTheme(row['ThemeID'])
        getData(row['DataID'])
        getGrid(row['GridID'])
    for index, row in df.iterrows():
        tutil.prepBlock(tutil.getReservedToken('BlockName',row), tutil.getReservedToken('CodeBlockID',row), {
                'ThemeID': tutil.getReservedToken('ThemeID',row),
                'DataID': tutil.getReservedToken('DataID',row),
                'GridID': tutil.getReservedToken('GridID',row)
        })
    for filename, blocks in tutil.trees['BLKTREE'].items():
        #print(filename)
        tutil.injectCode(filename, blocks)

In [18]:
COMMENT_FUNCT={
    'CS': CommentUtil.commentSlash,
    'TS': CommentUtil.commentSlash,
    'HTML': CommentUtil.commentHTML
}

In [19]:
RULES = {
    'UNIQ' : {
        0:{
            'InetAPI/Tts.Business.Logic/Aipm/AipmPortfolioScenarioManager.cs':[1, 2],
            'InetAPI/src/ApiGateways/Web.Bff.Admin/Controllers/Tts/Aipm/AipmPortfolioScenarioController.cs':[1],
            'InetAPI/Tts.Business.ViewModels/Aipm/PortfolioOptSummaryProjects.cs':[1],
            'InetAPI/Tts.Business.ViewModels/Aipm/AipmPortfolioResultChartModel.cs':[1],
            'InetUI/src/app/views/aipm/portfolio/portfolio.service.ts':[1,2],
            'InetUI/src/app/views/aipm/portfolio/portfolio.interface.ts':[1]
        }
    },
    'STARTMARKER':".*GENCODE:MARKER:.*:START.*",
    'ENDMARKER':".*GENCODE:MARKER:.*:END.*",
    'LEVEL_1_SUB':'##BLOCK##',
    'LEVEL_2_SUB':{
        'ThemeID': {
            '##(RESOLVE)\(([^#]+)\)##': funcResolve,
            '##(ITERATETHEMES)\(([^#]+)\)##': funcIterateThemes
        },
        'DataID': {
            '##(RESOLVEDATA)\(([^#]+)\)##': funcResolveData,
            '##(RESOLVEDATAESC)\(([^#]+)\)##': funcResolveDataEsc,
            '##(ITERATEDATA)\(([^#]+)\)##': funcIterateData
        },
        'GridID': {
            '##(RESOLVEGRID)\(([^#]+)\)##': funcResolveGrid,
            '##(RESOLVEGRIDESC)\(([^#]+)\)##': funcResolveGridEsc,
            '##(ITERATEGRID)\(([^#]+)\)##': funcIterateGrid
        },
        'VariableID':{
            '##(RESOLVEVAR)\(([^#]+)\)##': TreeUtil.funcResolveVar,
            '##(PRESERVE)\(([^#]+)\)##': TreeUtil.funcResolvePreserve
        }
    },
    'LEVEL_3_SUB':{
        'Param': {
            '(!!!PARAM:[^:]+:!!!)': TreeUtil.funcResolveParam,
            '(!!!PARAMDQUOTE:[^:]+:!!!)': TreeUtil.funcResolveParamDQuote,
            '(!!!PARAMDQUOTECOMMA:[^:]+:!!!)': TreeUtil.funcResolveParamDQuoteComma,
            '(!!!PARAMCOMMA:[^:]+:!!!)': TreeUtil.funcResolveParamComma
        }
    },
    'LEVEL_3_RULES':{
        "Replace": {
            "FieldDelimiter": TreeUtil.level3_rep_fd
        }
    },
    'ESCAPERULE':{
        'SQL': TreeUtil.padQuote
    },
    # NULLABLERULE is to cater for C# nullable declaration for datatype, adds a question mark at the end
    'NULLABLERULE':[
        'double',
        'int'
    ],
    'CONSTANTFUNC':{
        '!!!COMMA!!!': TreeUtil.funcResolveComma
    }
}

In [20]:
TREES = {
	'TPLTREE' : {},
	'BLKTREE' : {},
	'PHYTREE' : {},
	'THMTREE' : {},
	'GRDTREE' : {},
	'DATTREE' : {},
	'CLRTREE' : {},
	'VARTREE' : {},
	'NVPTREE' : {},
	'THMSUBTHM' : {},
	'DATSUBDAT' : {},
	'GRDSUBDET' : {},
	'uniqBlock' : {}
}

In [21]:
futil = FileUtil("Config/genCodeCfg.xls", COMMENT_FUNCT)

In [22]:
tutil = TreeUtil(futil, TREES, RULES)

cfgSheets = cfg.sheet_names
#print(cfgSheets)

In [23]:
runMainSetup("Config/genCodeCfg.xls", 'MainSetup', futil, tutil)

print(json.dumps(GRDTREE['PrioritisedProjectGrid']['Config']['PrioritisedProjectGridCFG'][1]['ActionRule'], indent=2))

print(json.dumps(BLKTREE['InetUI/src/app/views/aipm/portfolio/scenario-config/alternate-result/alternate-result.component.ts'], indent=2))